# Solveur VRPTW - Vehicle Routing Problem with Time Windows
## Projet ADEME - Optimisation de tournées de livraison

**Équipe:** CesiCDP  
**Date:** Novembre 2025

---

## Objectifs
- Résoudre VRPTW sur instances >1000 clients
- Gap moyen < 7% sur instances VRPLIB
- Contraintes: Fenêtres temporelles 
- Métaheuristique: Tabu Search + sa 
- Validation: 20 runs par instance

---
## 1. IMPORTS & CONFIGURATION

### Imports et Configuration

Ici, j'importe toutes les bibliothèques nécessaires pour mon solveur VRPTW :
- **numpy** et **pandas** pour les calculs numériques et la gestion des données
- **matplotlib** pour créer les visualisations des routes
- **dataclasses** pour définir mes structures de données (Customer, Vehicle, etc.)
- **random** et **math** pour les algorithmes métaheuristiques
- **time** pour mesurer les performances

Je configure aussi les chemins vers mes données d'instances VRPLIB.

# Explication de la cellule d'import et configuration

Dans cette cellule, j’ai chargé toutes les bibliothèques nécessaires à mon projet, à la fois pour la manipulation des données, les calculs numériques et la visualisation. J’ai également configuré les paramètres pour assurer la reproductibilité des résultats avec des graines fixes pour le générateur aléatoire. Enfin, j’ai défini un style graphique pour que les figures soient cohérentes et lisibles. Cette étape prépare l’environnement pour toutes les étapes suivantes du projet.

### Détails point par point :
- **Modules Python standard :** `os`, `re`, `time`, `math`, `random` pour manipulations diverses et calculs.
- **Gestion des copies :** `deepcopy` pour copier des objets complexes sans lien avec l’original.
- **Annotations de type :** `List`, `Tuple`, `Dict`, etc., pour plus de clarté.
- **Dataclasses :** simplifie la création de classes pour clients, véhicules, solutions.
- **Bibliothèques scientifiques :** `numpy`, `pandas` pour manipulations de données.
- **Visualisation :** `matplotlib.pyplot` et style `seaborn` si disponible.
- **Dictionnaires avec valeurs par défaut :** `defaultdict` pour regrouper ou compter facilement.
- **Reproductibilité :** `random.seed(42)` et `np.random.seed(42)` pour des résultats constants.
- **Configuration graphique :** taille des figures `(12,6)` et style graphique pour lisibilité.


In [12]:
import os
import re
import time
import math
import random
from copy import deepcopy
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass, field

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

# Configuration
random.seed(42)
np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid' if 'seaborn-v0_8-darkgrid' in plt.style.available else 'default')
plt.rcParams['figure.figsize'] = (12, 6)

print("Imports loaded successfully")

Imports loaded successfully


---
## 2. DATA LOADING (VRPLIB Format)

### Test du Chargement des Données

Maintenant que j'ai mes fonctions de parsing, je teste le chargement d'une instance réelle (C101 de Solomon).

C'est important de vérifier que :
- Le fichier se charge correctement
- Les données sont bien parsées (clients, dépôt, fenêtres temporelles)
- Les structures sont accessibles pour la suite du projet

# Explication des fonctions de parsing et de chargement d’instances

Cette cellule contient toutes les fonctions nécessaires pour **charger et transformer les instances CVRP et CVRPTW** en structures de données exploitables par mes algorithmes.

### Détail par fonction :

1. **`parse_solomon_instance(filepath)`**
   - Lit les fichiers Solomon (.txt) pour le CVRPTW.
   - Récupère le nombre de véhicules, leur capacité et toutes les données des clients (coordonnées, demande, fenêtres temporelles et temps de service).
   - Retourne un dictionnaire avec l’instance complète.

2. **`parse_vrp_instance(filepath)`**
   - Lit les fichiers TSPLIB (.vrp) pour le CVRP.
   - Récupère les coordonnées, les demandes et la capacité des véhicules.
   - Crée des fenêtres temporelles larges pour adapter au CVRPTW si nécessaire.
   - Estime le nombre minimal de véhicules nécessaires.

3. **`load_instance(filename)`**
   - Détecte automatiquement le type d’instance (.txt ou .vrp) et appelle la fonction de parsing correspondante.
   - Cherche les fichiers dans le dossier `DATA_PATH` et ses sous-dossiers.
   - Retourne un dictionnaire complet prêt à être utilisé dans l’algorithme.

4. **`load_solution(filename)`**
   - Charge le coût optimal depuis un fichier `.sol` si disponible.
   - Supporte différents formats de fichier solution.
   - Renvoie `None` si aucun fichier solution n’est trouvé.

### Points importants :
- `DATA_PATH` définit le chemin local vers les instances.
- Réindexation des clients à partir de 0 pour compatibilité Python.
- Gestion automatique des formats CVRP et CVRPTW.
- Préparation essentielle pour l’utilisation dans les algorithmes de recherche et de métaheuristiques.


In [32]:
# Chemin vers les instances locales
DATA_PATH = r"c:\Users\PICOS\Desktop\projet_RO\CVRPTWW\data"

def parse_solomon_instance(filepath: str) -> Dict:
    """
    Parse une instance au format Solomon (.txt)
    Format: C101.txt, R101.txt, RC101.txt
    """
    with open(filepath, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    
    instance = {
        'name': os.path.basename(filepath),
        'type': 'CVRPTW'
    }
    
    # Trouver la ligne avec NUMBER CAPACITY
    vehicle_line_idx = None
    customer_line_idx = None
    
    for i, line in enumerate(lines):
        if 'NUMBER' in line and 'CAPACITY' in line:
            vehicle_line_idx = i + 1  # Ligne suivante contient les valeurs
        elif 'CUST NO.' in line or 'CUSTOMER' in line:
            customer_line_idx = i + 1  # Les données clients commencent après
    
    # Parser véhicules
    if vehicle_line_idx:
        vehicle_info = lines[vehicle_line_idx].split()
        instance['num_vehicles'] = int(vehicle_info[0])
        instance['vehicle_capacity'] = int(vehicle_info[1])
    else:
        # Valeurs par défaut
        instance['num_vehicles'] = 25
        instance['vehicle_capacity'] = 200
    
    # Parser les clients
    customers = []
    if customer_line_idx:
        for line in lines[customer_line_idx:]:
            if not line.strip():  # Ignorer lignes vides
                continue
            
            parts = line.split()
            if len(parts) < 7:
                continue
            
            try:
                customer = {
                    'id': int(parts[0]),
                    'x': float(parts[1]),
                    'y': float(parts[2]),
                    'demand': int(parts[3]),
                    'ready_time': int(parts[4]),
                    'due_time': int(parts[5]),
                    'service_time': int(parts[6])
                }
                customers.append(customer)
            except ValueError:
                # Ignorer les lignes qui ne sont pas des données
                continue
    
    instance['customers'] = customers
    instance['num_customers'] = len(customers) - 1  # Sans le dépôt
    
    return instance


def parse_vrp_instance(filepath: str) -> Dict:
    """
    Parse une instance au format TSPLIB (.vrp)
    Format: A-n32-k5.vrp, X-n101-k25.vrp
    """
    with open(filepath, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    
    instance = {
        'name': os.path.basename(filepath),
        'type': 'CVRP'
    }
    
    coords = {}
    demands = {}
    section = None
    
    for line in lines:
        if 'CAPACITY' in line:
            instance['vehicle_capacity'] = int(line.split(':')[1].strip())
        elif 'NODE_COORD_SECTION' in line:
            section = 'coords'
        elif 'DEMAND_SECTION' in line:
            section = 'demands'
        elif 'DEPOT_SECTION' in line:
            section = 'depot'
        elif 'EOF' in line:
            break
        elif section == 'coords':
            parts = line.split()
            if len(parts) == 3:
                coords[int(parts[0])] = (float(parts[1]), float(parts[2]))
        elif section == 'demands':
            parts = line.split()
            if len(parts) == 2:
                demands[int(parts[0])] = int(parts[1])
    
    # Créer les clients avec fenêtres temporelles larges (pas de contrainte temporelle stricte)
    max_time = 1000
    customers = []
    for cid in sorted(coords.keys()):
        customer = {
            'id': cid - 1,  # Réindexer à partir de 0
            'x': coords[cid][0],
            'y': coords[cid][1],
            'demand': demands.get(cid, 0),
            'ready_time': 0,
            'due_time': max_time,
            'service_time': 10
        }
        customers.append(customer)
    
    instance['customers'] = customers
    instance['num_customers'] = len(customers) - 1
    
    # Estimer le nombre de véhicules nécessaires
    total_demand = sum(c['demand'] for c in customers if c['id'] > 0)
    instance['num_vehicles'] = max(5, (total_demand // instance['vehicle_capacity']) + 2)
    
    return instance


def load_instance(filename: str) -> Dict:
    """
    Charge une instance depuis le dossier data/
    Cherche d'abord à la racine, puis dans les sous-dossiers
    """
    # Chercher d'abord à la racine de data/
    filepath = os.path.join(DATA_PATH, filename)
    
    if os.path.exists(filepath):
        if filepath.endswith('.txt'):
            return parse_solomon_instance(filepath)
        elif filepath.endswith('.vrp'):
            return parse_vrp_instance(filepath)
    
    # Chercher dans les sous-dossiers
    for root, dirs, files in os.walk(DATA_PATH):
        if filename in files:
            filepath = os.path.join(root, filename)
            if filepath.endswith('.txt'):
                return parse_solomon_instance(filepath)
            elif filepath.endswith('.vrp'):
                return parse_vrp_instance(filepath)
    
    raise FileNotFoundError(f"Instance non trouvée: {filename} dans {DATA_PATH}")


def load_solution(filename: str) -> Optional[float]:
    """
    Charge le coût optimal depuis un fichier .sol
    Cherche dans data/ et ses sous-dossiers
    """
    # Chercher d'abord à la racine
    filepath = os.path.join(DATA_PATH, filename)
    
    if not os.path.exists(filepath):
        # Chercher dans les sous-dossiers
        found = False
        for root, dirs, files in os.walk(DATA_PATH):
            if filename in files:
                filepath = os.path.join(root, filename)
                found = True
                break
        
        if not found:
            print(f"Warning: Solution file not found: {filename}")
            return None
    
    with open(filepath, 'r') as f:
        lines = f.readlines()
        
        # Chercher la ligne avec "Cost" (format VRPLIB)
        for line in lines:
            line = line.strip()
            if line.startswith('Cost'):
                try:
                    # Format: "Cost 827.3"
                    cost = float(line.split()[1])
                    return cost
                except (ValueError, IndexError):
                    pass
        
        # Si pas de ligne "Cost", chercher première ligne avec un nombre
        for line in lines:
            line = line.strip()
            if line and line[0].isdigit():
                parts = line.split()
                try:
                    return float(parts[0])
                except (ValueError, IndexError):
                    pass
    
    return None


print("Data loading functions ready")

Data loading functions ready


### Sélection de l'Instance de Travail

Je choisis maintenant l'instance sur laquelle je vais travailler pour tout le notebook.

Cette instance sera utilisée pour :
- Tous les tests d'algorithmes
- Le run unique ALNS
- Les 20 runs de validation
- Toutes les visualisations

Je peux facilement changer l'instance ici pour tester sur différents problèmes.

In [34]:
# ============================================================================
# CHARGEMENT INSTANCE
# ============================================================================

# Sélectionner une instance
instance_name = "C101.txt"  # Modifier selon l'instance souhaitée

print(f"Loading instance: {instance_name}")
instance_data = load_instance(instance_name)
instance_obj = VRPInstance(instance_data)

print(f"\nInstance loaded:")
print(f"  Name: {instance_obj.name}")
print(f"  Type: {instance_obj.type}")
print(f"  Customers: {instance_obj.num_customers}")
print(f"  Vehicles: {instance_obj.num_vehicles}")

# Charger solution optimale si disponible
sol_filename = instance_name.replace('.txt', '.sol').replace('.vrp', '.sol')
optimal_cost = load_solution(sol_filename)

if optimal_cost:
    print(f"  Optimal cost: {optimal_cost:.2f}")
else:
    print(f"  Optimal cost: Unknown")

print(f"\nInstance ready for optimization")

Loading instance: C101.txt

Instance loaded:
  Name: C101.txt
  Type: CVRPTW
  Customers: 100
  Vehicles: 25
  Optimal cost: 827.30

Instance ready for optimization


---
## 3. BASE CLASSES

### Classe Customer

Je commence par définir la classe Customer qui représente un client dans mon problème CVRPTW.

Chaque client possède :
- Un identifiant unique
- Des coordonnées (x, y) pour calculer les distances
- Une demande (quantité à livrer)
- Une fenêtre temporelle [ready_time, due_date]
- Un temps de service

Ces attributs sont essentiels pour vérifier les contraintes de capacité et de temps.

# Classes de base pour le VRP

Cette cellule définit les **objets principaux** qui servent à représenter les données de mon projet VRP/CVRPTW de manière structurée.

### 1. Customer
- Représente un client avec ses caractéristiques :
  - `id`, coordonnées (`x`, `y`)
  - Demande (`demand`)
  - Fenêtres temporelles (`ready_time`, `due_time`)
  - Temps de service (`service_time`)
- Utilise `@dataclass` pour simplifier la création et la gestion des objets.

### 2. Vehicle
- Représente un véhicule de livraison :
  - `id`, capacité (`capacity`)
  - `cost_per_km` pour prendre en compte la flotte hétérogène si nécessaire.

### 3. VRPInstance
- Représente une instance complète du problème VRP :
  - Crée les objets `Customer` et `Vehicle` à partir des données chargées.
  - Identifie le dépôt (`depot = customers[0]`).
  - Construit une **matrice des distances euclidiennes** pour tous les clients.
  - Méthode `distance(i, j)` pour accéder rapidement à la distance entre deux clients.

### Points importants
- La matrice des distances est symétrique et pré-calculée pour optimiser les performances.
- La structure orientée objet rend le code plus modulable et facile à utiliser avec les métaheuristiques (Clarke-Wright, ALNS, VND, etc.).


In [14]:
@dataclass
class Customer:
    """Représente un client avec ses caractéristiques"""
    id: int
    x: float
    y: float
    demand: int
    ready_time: int
    due_time: int
    service_time: int


@dataclass
class Vehicle:
    """Représente un véhicule avec sa capacité"""
    id: int
    capacity: int
    cost_per_km: float = 1.0  # Pour flotte hétérogène


class VRPInstance:
    """Instance complète du problème VRP"""
    
    def __init__(self, data: Dict):
        self.name = data['name']
        self.type = data['type']
        self.num_customers = data['num_customers']
        self.vehicle_capacity = data['vehicle_capacity']
        self.num_vehicles = data['num_vehicles']
        
        # Créer les objets Customer
        self.customers = [
            Customer(**c) for c in data['customers']
        ]
        
        self.depot = self.customers[0]
        
        # Créer les véhicules
        self.vehicles = [
            Vehicle(id=v, capacity=self.vehicle_capacity)
            for v in range(self.num_vehicles)
        ]
        
        # Matrice des distances
        self.distance_matrix = self._build_distance_matrix()
        
    def _build_distance_matrix(self) -> np.ndarray:
        """Construit la matrice des distances euclidiennes"""
        n = len(self.customers)
        matrix = np.zeros((n, n))
        
        for i in range(n):
            for j in range(i + 1, n):
                dx = self.customers[i].x - self.customers[j].x
                dy = self.customers[i].y - self.customers[j].y
                dist = math.sqrt(dx * dx + dy * dy)
                matrix[i][j] = dist
                matrix[j][i] = dist
        
        return matrix
    
    def distance(self, i: int, j: int) -> float:
        """Return distance between two customers"""
        return self.distance_matrix[i][j]


print("Base classes loaded: Customer, Vehicle, VRPInstance")

Base classes loaded: Customer, Vehicle, VRPInstance


# Classe Solution (VRPTW)

Cette cellule définit la classe **`Solution`**, qui représente une solution candidate pour le VRPTW (Vehicle Routing Problem with Time Windows).

### Attributs principaux
- `routes`: liste des routes (chaque route = liste d'IDs clients, sans le dépôt).
- `cost`: coût total de la solution, incluant les pénalités.
- `distance`: distance totale parcourue.
- `time_violations`: accumule les écarts aux fenêtres temporelles.
- **Pas de contrainte de capacité** dans VRPTW.

### Méthodes principales
- `add_route(route)`: ajoute une route à la solution.
- `calculate_cost()`: calcule le coût total = distance + pénalités pour violations temporelles.
- `_route_distance(route)`: distance totale d'une route incluant le dépôt.
- `_check_time_violation(route)`: vérifie les fenêtres temporelles (attente autorisée).
- `is_feasible()`: indique si la solution est réalisable (pas de violations temporelles).
- `get_unvisited_customers()`: retourne les clients non visités.
- `copy()`: crée une copie profonde de la solution.

### Points importants
- Les violations de fenêtres temporelles sont fortement pénalisées pour guider les algorithmes.
- La classe gère la **distance et les contraintes temporelles** dans une structure unique.
- Permet de manipuler facilement des solutions, vérifier leur faisabilité et calculer le coût pour les métaheuristiques (Tabu Search, Clarke-Wright, etc.).

### Test des Classes de Base

Je teste maintenant mes classes de base avec l'instance C101 chargée précédemment.

Ce test me permet de vérifier que :
- L'instance se crée correctement avec tous les clients
- La matrice de distances est bien calculée
- Je peux créer une solution vide
- Les méthodes de base fonctionnent (ajout de routes, calcul de coût)

In [16]:
class Solution:
    """Solution representation with constraint validation (VRPTW - time windows)"""
    
    def __init__(self, instance: VRPInstance):
        self.instance = instance
        self.routes: List[List[int]] = []  # Liste de routes (IDs clients sans dépôt)
        self.cost: float = float('inf')
        self.distance: float = 0.0
        self.time_violations: float = 0.0
        
    def add_route(self, route: List[int]):
        """Ajoute une route à la solution"""
        if route:
            self.routes.append(route)
    
    def calculate_cost(self) -> float:
        """
        Calcule le coût total avec pénalités pour violations
        Coût = Distance + 10000 * violations_temps
        (Pas de contrainte de capacité pour VRPTW)
        """
        self.distance = 0.0
        self.time_violations = 0.0
        
        for route in self.routes:
            if not route:
                continue
            
            # Distance de la route
            self.distance += self._route_distance(route)
            
            # Vérification contraintes temporelles uniquement
            self.time_violations += self._check_time_violation(route)
        
        # Pénalité forte pour violations temporelles
        penalty = 10000 * self.time_violations
        self.cost = self.distance + penalty
        
        return self.cost
    
    def _route_distance(self, route: List[int]) -> float:
        """Calcule la distance totale d'une route (incluant dépôt)"""
        if not route:
            return 0.0
        
        dist = 0.0
        
        # Dépôt (0) -> premier client
        dist += self.instance.distance(0, route[0])
        
        # Entre clients consécutifs
        for i in range(len(route) - 1):
            dist += self.instance.distance(route[i], route[i + 1])
        
        # Dernier client -> dépôt (0)
        dist += self.instance.distance(route[-1], 0)
        
        return dist
    
    def _check_time_violation(self, route: List[int]) -> float:
        """
        Vérifie les fenêtres temporelles avec ATTENTE AUTORISÉE
        Retourne la somme des retards (arrivée > due_time)
        """
        if not route:
            return 0.0
        
        current_time = 0.0
        violation = 0.0
        
        # Départ du dépôt
        prev_customer = 0
        
        for customer_id in route:
            customer = self.instance.customers[customer_id]
            
            # Temps de trajet
            travel_time = self.instance.distance(prev_customer, customer_id)
            arrival_time = current_time + travel_time
            
            # Gestion fenêtre temporelle
            if arrival_time < customer.ready_time:
                # Arrivée trop tôt → ATTENTE (pas de violation)
                current_time = customer.ready_time
            elif arrival_time > customer.due_time:
                # Arrivée trop tard → VIOLATION
                violation += (arrival_time - customer.due_time)
                current_time = arrival_time
            else:
                # Arrivée dans la fenêtre
                current_time = arrival_time
            
            # Ajouter temps de service
            current_time += customer.service_time
            prev_customer = customer_id
        
        # Retour au dépôt
        travel_to_depot = self.instance.distance(prev_customer, 0)
        arrival_depot = current_time + travel_to_depot
        depot_due = self.instance.depot.due_time
        
        if arrival_depot > depot_due:
            violation += (arrival_depot - depot_due)
        
        return violation
    
    def is_feasible(self) -> bool:
        """Vérifie si la solution est réalisable (sans violations temporelles)"""
        self.calculate_cost()
        return self.time_violations == 0
    
    def get_unvisited_customers(self) -> List[int]:
        """Retourne la liste des clients non visités"""
        visited = set()
        for route in self.routes:
            visited.update(route)
        
        all_customers = set(range(1, len(self.instance.customers)))
        return list(all_customers - visited)
    
    def copy(self):
        """Crée une copie profonde de la solution"""
        sol = Solution(self.instance)
        sol.routes = [route[:] for route in self.routes]
        sol.cost = self.cost
        sol.distance = self.distance
        sol.time_violations = self.time_violations
        return sol


print("Solution class loaded (VRPTW version - no capacity constraints)")

Solution class loaded (VRPTW version - no capacity constraints)


---
## 4. SOLUTION INITIALE

### Solution Initiale Simple

Pour démarrer le Tabu Search, je crée une solution initiale simple :
- Chaque client a sa propre route (dépôt → client → dépôt)
- C'est une solution triviale mais toujours faisable
- Le Tabu Search va ensuite optimiser en fusionnant et réorganisant les routes

Pas besoin d'algorithme complexe ici, la métaheuristique fera le travail.

In [17]:
def create_initial_solution(instance: VRPInstance) -> Solution:
    """
    Crée une solution initiale simple pour VRPTW
    
    Principe :
    - Chaque client a sa propre route: dépôt → client → dépôt
    - Toujours faisable (pas de contraintes de capacité en VRPTW)
    - Le Tabu Search optimisera cette solution
    """
    
    print("🔧 Génération solution initiale (routes individuelles)...")
    solution = Solution(instance)
    
    # Créer une route pour chaque client
    for customer_id in range(1, len(instance.customers)):
        solution.add_route([customer_id])
    
    solution.calculate_cost()
    print(f"✅ Solution initiale créée: {len(solution.routes)} routes, coût={solution.cost:.2f}")
    
    return solution


def _is_route_time_feasible(instance: VRPInstance, route: List[int]) -> bool:
    """
    Vérifie rapidement si une route respecte les fenêtres temporelles
    Retourne False si violation détectée
    """
    current_time = 0.0
    prev_customer = 0
    
    for customer_id in route:
        customer = instance.customers[customer_id]
        
        # Temps de trajet
        travel_time = instance.distance(prev_customer, customer_id)
        arrival_time = current_time + travel_time
        
        # Si arrivée après la fenêtre, infaisable
        if arrival_time > customer.due_time:
            return False
        
        # Attente si trop tôt
        if arrival_time < customer.ready_time:
            current_time = customer.ready_time
        else:
            current_time = arrival_time
        
        # Service
        current_time += customer.service_time
        prev_customer = customer_id
    
    return True


print("Initial solution generator loaded (simple individual routes)")

Initial solution generator loaded (simple individual routes)


---
## 5. VND (Variable Neighborhood Descent)

### Opérateurs de Recherche Locale

J'implémente trois opérateurs de recherche locale pour améliorer les solutions :

**1. 2-opt intra-route** : Inverse un segment dans une route
   - Teste tous les segments possibles
   - Garde l'amélioration si elle existe

**2. Relocate** : Déplace un client vers une autre position
   - Teste tous les clients dans toutes les positions (intra et inter-routes)
   - Très efficace pour ajuster les routes

**3. Swap** : Échange deux clients
   - Teste tous les échanges possibles (intra et inter-routes)
   - Permet de mieux équilibrer les routes

Ces opérateurs seront utilisés dans VND pour intensifier la recherche.

### VND - Variable Neighborhood Descent

VND est une méthode d'intensification qui applique séquentiellement les opérateurs de recherche locale :
- Tant qu'une amélioration est trouvée, on continue
- On parcourt les 3 opérateurs : 2-opt, relocate, swap
- Utilisé périodiquement dans Tabu Search et SA pour intensifier

In [18]:
def vnd(solution: Solution, max_iterations: int = 100) -> Solution:
    """
    VND (Variable Neighborhood Descent) - Intensification locale
    
    Principe:
    - Applique séquentiellement les 3 opérateurs (2-opt, relocate, swap)
    - Continue tant qu'une amélioration est trouvée
    - S'arrête quand plus aucune amélioration n'est possible
    
    Args:
        solution: Solution à améliorer (modifiée in-place)
        max_iterations: Nombre max d'itérations
    
    Returns:
        Solution améliorée
    """
    improved = True
    iteration = 0
    initial_cost = solution.calculate_cost()
    
    while improved and iteration < max_iterations:
        improved = False
        
        # 1. 2-opt intra-route sur chaque route
        for route_idx in range(len(solution.routes)):
            if two_opt_intra_route(solution, route_idx):
                solution.calculate_cost()
                improved = True
        
        # 2. Relocate
        if relocate(solution):
            solution.calculate_cost()
            improved = True
        
        # 3. Swap
        if swap(solution):
            solution.calculate_cost()
            improved = True
        
        iteration += 1
    
    final_cost = solution.calculate_cost()
    improvement = initial_cost - final_cost
    
    return solution


def _is_insertion_feasible(instance: VRPInstance, route: List[int]) -> bool:
    """
    Vérifie si une route est faisable (fenêtres temporelles uniquement)
    Utilisé par les opérateurs de recherche locale
    """
    return _is_route_time_feasible(instance, route)


print("VND (Variable Neighborhood Descent) loaded")

VND (Variable Neighborhood Descent) loaded


In [19]:
def two_opt_intra_route(solution: Solution, route_idx: int) -> bool:
    """
    2-OPT intra-route: Inverse un segment dans une route
    
    Principe:
    Pour une route [... i-1, i, ..., j, j+1 ...], on inverse le segment [i, ..., j]
    Résultat: [... i-1, j, ..., i, j+1 ...]
    
    Returns:
        True si amélioration trouvée
    """
    route = solution.routes[route_idx]
    n = len(route)
    
    if n < 2:
        return False
    
    improved = False
    best_delta = 0
    best_i, best_j = -1, -1
    
    # Tester toutes les paires (i, j)
    for i in range(n - 1):
        for j in range(i + 1, n):
            # Créer nouvelle route avec segment inversé
            new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
            
            # Vérifier faisabilité
            if not _is_insertion_feasible(solution.instance, new_route):
                continue
            
            # Calculer gain
            old_cost = solution._route_distance(route)
            new_cost = solution._route_distance(new_route)
            delta = new_cost - old_cost
            
            if delta < best_delta:
                best_delta = delta
                best_i, best_j = i, j
                improved = True
    
    # Appliquer la meilleure amélioration
    if improved:
        solution.routes[route_idx] = (route[:best_i] + 
                                      route[best_i:best_j+1][::-1] + 
                                      route[best_j+1:])
    
    return improved


def relocate(solution: Solution) -> bool:
    """
    RELOCATE: Déplace un client d'une route vers une autre position
    
    Teste:
    - Déplacements intra-route (changement de position dans la même route)
    - Déplacements inter-route (vers une autre route)
    
    Returns:
        True si amélioration trouvée
    """
    improved = False
    best_delta = 0
    best_move = None
    
    # Pour chaque client
    for from_route_idx, from_route in enumerate(solution.routes):
        for from_pos, customer_id in enumerate(from_route):
            
            # Essayer de le déplacer dans chaque route (y compris la même)
            for to_route_idx, to_route in enumerate(solution.routes):
                for to_pos in range(len(to_route) + 1):
                    
                    # Skip si même position
                    if from_route_idx == to_route_idx and to_pos == from_pos:
                        continue
                    
                    # Créer nouvelles routes
                    new_from_route = from_route[:from_pos] + from_route[from_pos+1:]
                    
                    if from_route_idx == to_route_idx:
                        # Intra-route: ajuster position si nécessaire
                        adj_to_pos = to_pos if to_pos < from_pos else to_pos - 1
                        new_to_route = (new_from_route[:adj_to_pos] + 
                                       [customer_id] + 
                                       new_from_route[adj_to_pos:])
                    else:
                        # Inter-route
                        new_to_route = to_route[:to_pos] + [customer_id] + to_route[to_pos:]
                    
                    # Vérifier faisabilité
                    if new_from_route and not _is_insertion_feasible(solution.instance, new_from_route):
                        continue
                    if not _is_insertion_feasible(solution.instance, new_to_route):
                        continue
                    
                    # Calculer gain
                    old_cost = (solution._route_distance(from_route) + 
                               (solution._route_distance(to_route) if from_route_idx != to_route_idx else 0))
                    
                    new_cost = (solution._route_distance(new_from_route) if new_from_route else 0) + \
                               (solution._route_distance(new_to_route) if from_route_idx != to_route_idx else 0)
                    
                    if from_route_idx == to_route_idx:
                        new_cost = solution._route_distance(new_to_route)
                    
                    delta = new_cost - old_cost
                    
                    if delta < best_delta:
                        best_delta = delta
                        best_move = (from_route_idx, from_pos, to_route_idx, to_pos, 
                                    new_from_route, new_to_route)
                        improved = True
    
    # Appliquer le meilleur mouvement
    if improved and best_move:
        from_route_idx, from_pos, to_route_idx, to_pos, new_from_route, new_to_route = best_move
        
        if from_route_idx == to_route_idx:
            solution.routes[from_route_idx] = new_to_route
        else:
            if new_from_route:
                solution.routes[from_route_idx] = new_from_route
            else:
                solution.routes.pop(from_route_idx)
                if to_route_idx > from_route_idx:
                    to_route_idx -= 1
            
            if to_route_idx < len(solution.routes):
                solution.routes[to_route_idx] = new_to_route
            else:
                solution.routes.append(new_to_route)
    
    return improved


def swap(solution: Solution) -> bool:
    """
    SWAP: Échange deux clients entre deux routes
    
    Teste tous les échanges possibles entre:
    - Clients de la même route (intra-route swap)
    - Clients de routes différentes (inter-route swap)
    
    Returns:
        True si amélioration trouvée
    """
    improved = False
    best_delta = 0
    best_swap = None
    
    # Pour chaque paire de routes
    for route1_idx in range(len(solution.routes)):
        for route2_idx in range(route1_idx, len(solution.routes)):
            route1 = solution.routes[route1_idx]
            route2 = solution.routes[route2_idx]
            
            # Pour chaque paire de clients
            for pos1, cust1 in enumerate(route1):
                start_pos2 = pos1 + 1 if route1_idx == route2_idx else 0
                
                for pos2 in range(start_pos2, len(route2)):
                    cust2 = route2[pos2]
                    
                    # Créer nouvelles routes avec swap
                    new_route1 = route1[:pos1] + [cust2] + route1[pos1+1:]
                    new_route2 = route2[:pos2] + [cust1] + route2[pos2+1:]
                    
                    # Vérifier faisabilité
                    if not _is_insertion_feasible(solution.instance, new_route1):
                        continue
                    if not _is_insertion_feasible(solution.instance, new_route2):
                        continue
                    
                    # Calculer gain
                    old_cost = solution._route_distance(route1)
                    if route1_idx != route2_idx:
                        old_cost += solution._route_distance(route2)
                    
                    new_cost = solution._route_distance(new_route1)
                    if route1_idx != route2_idx:
                        new_cost += solution._route_distance(new_route2)
                    
                    delta = new_cost - old_cost
                    
                    if delta < best_delta:
                        best_delta = delta
                        best_swap = (route1_idx, route2_idx, new_route1, new_route2)
                        improved = True
    
    # Appliquer le meilleur swap
    if improved and best_swap:
        route1_idx, route2_idx, new_route1, new_route2 = best_swap
        solution.routes[route1_idx] = new_route1
        if route1_idx != route2_idx:
            solution.routes[route2_idx] = new_route2
    
    return improved


print("Local search operators loaded")

Local search operators loaded


---
## 6. TABU SEARCH

### Classe Tabu Search

J'implémente la classe Tabu Search pour le VRPTW.

**Architecture** :
1. **Initialisation** : Solution simple (routes individuelles)
2. **Boucle Tabu Search** :
   - Explorer le voisinage (relocate, swap, 2-opt)
   - Sélectionner le meilleur mouvement non-tabou
   - Critère d'aspiration: accepter un mouvement tabou si amélioration globale
   - Ajouter le mouvement à la liste taboue
   - Mettre à jour la meilleure solution
3. **Intensification** : Application périodique de VND
4. **Diversification** : Perturbations si stagnation

**Paramètres** :
- `max_iterations` : nombre max d'itérations Tabu Search
- `tabu_tenure` : durée de vie dans la liste taboue
- `neighborhood_size` : nombre de voisins à explorer

In [20]:
class TabuSearch:
    """
    Tabu Search pour VRPTW (Vehicle Routing Problem with Time Windows)
    
    Principe:
    1. Partir d'une solution initiale simple (routes individuelles)
    2. À chaque itération:
       - Explorer le voisinage (relocate, swap, 2-opt)
       - Sélectionner le meilleur mouvement non-tabou
       - Critère d'aspiration: accepter un mouvement tabou si amélioration
       - Ajouter le mouvement à la liste taboue
    3. Appliquer VND périodiquement pour intensification
    """
    
    def __init__(self, instance: VRPInstance, 
                 max_iterations: int = 1000,
                 max_time: float = 300,
                 tabu_tenure: int = 10,
                 neighborhood_size: int = 100):
        
        self.instance = instance
        self.max_iterations = max_iterations
        self.max_time = max_time
        self.tabu_tenure = tabu_tenure
        self.neighborhood_size = neighborhood_size
        
        # Liste taboue: dictionnaire {move: iteration_expiration}
        self.tabu_list = {}
        
        # Historique
        self.cost_history = []
        self.best_cost_history = []
        
    def solve(self, verbose: bool = True) -> Solution:
        """
        Algorithme Tabu Search principal
        """
        start_time = time.time()
        
        if verbose:
            print("\n" + "-" * 70)
            print("TABU SEARCH OPTIMIZATION")
            print("-" * 70)
        
        # 1. Solution initiale simple
        current = create_initial_solution(self.instance)
        best = current.copy()
        
        if verbose:
            print(f"\nStarting Tabu Search from initial solution")
            print(f"  Initial cost: {best.cost:.2f}")
            print(f"  Initial routes: {len(best.routes)}")
        
        iteration = 0
        iterations_without_improvement = 0
        
        if verbose:
            print(f"\nParameters:")
            print(f"  Max iterations: {self.max_iterations}")
            print(f"  Max time: {self.max_time}s")
            print(f"  Tabu tenure: {self.tabu_tenure}")
            print(f"  Neighborhood size: {self.neighborhood_size}")
            print(f"\nOptimizing...")
        
        # Boucle principale Tabu Search
        while iteration < self.max_iterations and (time.time() - start_time) < self.max_time:
            
            # Explorer le voisinage
            neighbors = self._explore_neighborhood(current)
            
            if not neighbors:
                # Diversification si aucun voisin
                if verbose and iteration % 50 == 0:
                    print(f"  Iter {iteration:4d} | No feasible neighbors, diversifying...")
                current = self._diversify(best)
                iteration += 1
                continue
            
            # Sélectionner le meilleur mouvement
            best_neighbor = None
            best_move = None
            best_neighbor_cost = float('inf')
            
            for neighbor, move in neighbors:
                neighbor.calculate_cost()
                
                # Vérifier si tabou
                is_tabu = self._is_tabu(move, iteration)
                
                # Critère d'aspiration
                aspiration = neighbor.cost < best.cost
                
                if (not is_tabu or aspiration) and neighbor.cost < best_neighbor_cost:
                    best_neighbor = neighbor
                    best_move = move
                    best_neighbor_cost = neighbor.cost
            
            if best_neighbor is None:
                # Tous tabous, prendre le meilleur quand même
                best_neighbor, best_move = min(neighbors, key=lambda x: x[0].calculate_cost() or x[0].cost)
            
            # Appliquer le mouvement
            current = best_neighbor
            self._add_tabu(best_move, iteration)
            
            # Mise à jour meilleure solution
            if current.cost < best.cost:
                best = current.copy()
                iterations_without_improvement = 0
                if verbose and (iteration % 50 == 0 or iteration < 10):
                    print(f"  Iter {iteration:4d} | New best: {best.cost:.2f} | Routes: {len(best.routes)}")
            else:
                iterations_without_improvement += 1
            
            # VND périodique
            if iteration > 0 and iteration % 100 == 0 and len(best.routes) > 0:
                best_copy = best.copy()
                vnd(best_copy, max_iterations=20)
                if best_copy.cost < best.cost:
                    best = best_copy
                    current = best.copy()
                    if verbose:
                        print(f"  Iter {iteration:4d} | VND improved to: {best.cost:.2f}")
            
            # Diversification si stagnation
            if iterations_without_improvement > 50:
                if verbose:
                    print(f"  Iter {iteration:4d} | Stagnation, diversifying...")
                current = self._diversify(best)
                iterations_without_improvement = 0
                self.tabu_list.clear()  # Reset tabu list
            
            # Historique
            self.cost_history.append(current.cost)
            self.best_cost_history.append(best.cost)
            
            # Nettoyer liste taboue
            self._clean_tabu_list(iteration)
            
            iteration += 1
        
        elapsed = time.time() - start_time
        
        if verbose:
            print("\n" + "-" * 70)
            print("TABU SEARCH COMPLETED")
            print("-" * 70)
            print(f"\nResults:")
            print(f"  Iterations: {iteration}")
            print(f"  Time: {elapsed:.2f}s")
            print(f"  Best cost: {best.cost:.2f}")
            print(f"  Best distance: {best.distance:.2f}")
            print(f"  Routes: {len(best.routes)}")
            print(f"  Feasible: {best.is_feasible()}")
        
        return best
    
    def _explore_neighborhood(self, solution: Solution) -> List[Tuple[Solution, Tuple]]:
        """
        Explore le voisinage: relocate, swap, 2-opt
        """
        neighbors = []
        count = 0
        max_count = self.neighborhood_size
        
        # 1. Relocate: déplacer un client vers une autre position
        for route_idx, route in enumerate(solution.routes):
            if count >= max_count:
                break
            for pos in range(len(route)):
                if count >= max_count:
                    break
                customer = route[pos]
                
                # Vers d'autres routes
                for target_route_idx in range(len(solution.routes)):
                    if count >= max_count:
                        break
                    if target_route_idx == route_idx:
                        continue
                    
                    for target_pos in range(len(solution.routes[target_route_idx]) + 1):
                        neighbor = solution.copy()
                        
                        # Relocate
                        customer_id = neighbor.routes[route_idx].pop(pos)
                        neighbor.routes[target_route_idx].insert(target_pos, customer_id)
                        neighbor.routes = [r for r in neighbor.routes if r]
                        
                        if self._is_neighbor_feasible(neighbor):
                            move = ('relocate', customer, route_idx, target_route_idx, target_pos)
                            neighbors.append((neighbor, move))
                            count += 1
        
        # 2. Swap: échanger deux clients
        for i in range(len(solution.routes)):
            if count >= max_count:
                break
            for j in range(i, len(solution.routes)):
                if count >= max_count:
                    break
                
                route1 = solution.routes[i]
                route2 = solution.routes[j]
                
                for pos1 in range(len(route1)):
                    if count >= max_count:
                        break
                    start_pos2 = 0 if i != j else pos1 + 1
                    for pos2 in range(start_pos2, len(route2)):
                        if count >= max_count:
                            break
                        
                        neighbor = solution.copy()
                        neighbor.routes[i][pos1], neighbor.routes[j][pos2] = \
                            neighbor.routes[j][pos2], neighbor.routes[i][pos1]
                        
                        if self._is_neighbor_feasible(neighbor):
                            move = ('swap', route1[pos1], route2[pos2], i, j, pos1, pos2)
                            neighbors.append((neighbor, move))
                            count += 1
        
        # 3. 2-opt intra-route
        for route_idx, route in enumerate(solution.routes):
            if count >= max_count:
                break
            if len(route) < 4:
                continue
            
            for i in range(len(route) - 2):
                if count >= max_count:
                    break
                for j in range(i + 2, min(i + 5, len(route))):  # Limiter la taille
                    neighbor = solution.copy()
                    neighbor.routes[route_idx][i+1:j+1] = list(reversed(neighbor.routes[route_idx][i+1:j+1]))
                    
                    if self._is_neighbor_feasible(neighbor):
                        move = ('2opt', route_idx, i, j)
                        neighbors.append((neighbor, move))
                        count += 1
        
        return neighbors
    
    def _is_neighbor_feasible(self, solution: Solution) -> bool:
        """Vérification rapide de faisabilité temporelle"""
        for route in solution.routes:
            if route and not _is_route_time_feasible(self.instance, route):
                return False
        return True
    
    def _is_tabu(self, move: Tuple, current_iteration: int) -> bool:
        """Vérifie si un mouvement est tabou"""
        return move in self.tabu_list and self.tabu_list[move] > current_iteration
    
    def _add_tabu(self, move: Tuple, current_iteration: int):
        """Ajoute un mouvement à la liste taboue"""
        self.tabu_list[move] = current_iteration + self.tabu_tenure
    
    def _clean_tabu_list(self, current_iteration: int):
        """Nettoie la liste taboue"""
        expired = [move for move, expiry in self.tabu_list.items() if expiry <= current_iteration]
        for move in expired:
            del self.tabu_list[move]
    
    def _diversify(self, solution: Solution) -> Solution:
        """Diversification: perturbation aléatoire"""
        diversified = solution.copy()
        
        num_moves = min(10, len(diversified.routes) * 2)
        
        for _ in range(num_moves):
            if len(diversified.routes) < 2:
                break
            
            # Mouvements aléatoires
            if random.random() < 0.5 and len(diversified.routes) >= 2:
                # Swap aléatoire
                idx1, idx2 = random.sample(range(len(diversified.routes)), 2)
                if diversified.routes[idx1] and diversified.routes[idx2]:
                    pos1 = random.randint(0, len(diversified.routes[idx1]) - 1)
                    pos2 = random.randint(0, len(diversified.routes[idx2]) - 1)
                    diversified.routes[idx1][pos1], diversified.routes[idx2][pos2] = \
                        diversified.routes[idx2][pos2], diversified.routes[idx1][pos1]
            else:
                # Relocate aléatoire
                idx1 = random.randint(0, len(diversified.routes) - 1)
                if diversified.routes[idx1]:
                    idx2 = random.randint(0, len(diversified.routes) - 1)
                    pos1 = random.randint(0, len(diversified.routes[idx1]) - 1)
                    customer = diversified.routes[idx1].pop(pos1)
                    pos2 = random.randint(0, len(diversified.routes[idx2]))
                    diversified.routes[idx2].insert(pos2, customer)
        
        diversified.routes = [r for r in diversified.routes if r]
        diversified.calculate_cost()
        return diversified


print("Tabu Search class loaded")

Tabu Search class loaded


### Classe Simulated Annealing (Amélioration)

Maintenant j'implémente le Simulated Annealing pour améliorer la solution du Tabu Search.

**Principe** :
1. Partir de la solution obtenue par Tabu Search
2. Appliquer des perturbations aléatoires
3. Accepter les dégradations avec une probabilité décroissante (température)
4. Permet de sortir des optimums locaux

**Complémentarité** :
- **Tabu Search** : exploration systématique, construction de bonnes solutions
- **Simulated Annealing** : exploration stochastique, raffinement final

C'est une approche hybride qui combine les forces des deux métaheuristiques.

In [21]:
class SimulatedAnnealing:
    """
    Simulated Annealing pour améliorer une solution VRPTW
    
    Utilisé après Tabu Search pour raffiner la solution
    """
    
    def __init__(self, instance: VRPInstance,
                 max_iterations: int = 1000,
                 temperature_init: float = 100.0,
                 cooling_rate: float = 0.995,
                 max_time: float = 120):
        
        self.instance = instance
        self.max_iterations = max_iterations
        self.temperature_init = temperature_init
        self.cooling_rate = cooling_rate
        self.max_time = max_time
        
        # Historique
        self.cost_history = []
        self.best_cost_history = []
    
    def improve(self, initial_solution: Solution, verbose: bool = True) -> Solution:
        """
        Améliore une solution existante avec Simulated Annealing
        """
        start_time = time.time()
        
        if verbose:
            print("\n" + "-" * 70)
            print("SIMULATED ANNEALING REFINEMENT")
            print("-" * 70)
        
        current = initial_solution.copy()
        best = current.copy()
        
        if verbose:
            print(f"\nStarting from Tabu Search solution")
            print(f"  Initial cost: {best.cost:.2f}")
            print(f"  Initial routes: {len(best.routes)}")
        
        temperature = self.temperature_init
        iteration = 0
        accepted_moves = 0
        rejected_moves = 0
        
        if verbose:
            print(f"\nParameters:")
            print(f"  Max iterations: {self.max_iterations}")
            print(f"  Initial temperature: {self.temperature_init}")
            print(f"  Cooling rate: {self.cooling_rate}")
            print(f"\nRefining...")
        
        while iteration < self.max_iterations and (time.time() - start_time) < self.max_time:
            
            # Générer un voisin aléatoire
            neighbor = self._generate_neighbor(current)
            
            if neighbor is None:
                iteration += 1
                continue
            
            neighbor.calculate_cost()
            delta = neighbor.cost - current.cost
            
            # Critère d'acceptation
            if delta < 0:
                # Amélioration: toujours accepter
                current = neighbor
                accepted_moves += 1
                
                if current.cost < best.cost:
                    best = current.copy()
                    if verbose and iteration % 100 == 0:
                        print(f"  Iter {iteration:4d} | New best: {best.cost:.2f} | T={temperature:.2f}")
            
            elif temperature > 0:
                # Dégradation: accepter avec probabilité
                probability = math.exp(-delta / temperature)
                if random.random() < probability:
                    current = neighbor
                    accepted_moves += 1
                else:
                    rejected_moves += 1
            else:
                rejected_moves += 1
            
            # Refroidissement
            temperature *= self.cooling_rate
            
            # Historique
            self.cost_history.append(current.cost)
            self.best_cost_history.append(best.cost)
            
            # VND périodique
            if iteration > 0 and iteration % 200 == 0:
                best_copy = best.copy()
                vnd(best_copy, max_iterations=10)
                if best_copy.cost < best.cost:
                    best = best_copy
                    current = best.copy()
                    if verbose:
                        print(f"  Iter {iteration:4d} | VND improved to: {best.cost:.2f}")
            
            iteration += 1
        
        elapsed = time.time() - start_time
        acceptance_rate = 100 * accepted_moves / max(1, accepted_moves + rejected_moves)
        
        if verbose:
            print("\n" + "-" * 70)
            print("SIMULATED ANNEALING COMPLETED")
            print("-" * 70)
            print(f"\nResults:")
            print(f"  Iterations: {iteration}")
            print(f"  Time: {elapsed:.2f}s")
            print(f"  Accepted moves: {accepted_moves}")
            print(f"  Rejected moves: {rejected_moves}")
            print(f"  Acceptance rate: {acceptance_rate:.1f}%")
            print(f"  Best cost: {best.cost:.2f}")
            print(f"  Best distance: {best.distance:.2f}")
            print(f"  Routes: {len(best.routes)}")
            print(f"  Feasible: {best.is_feasible()}")
            
            improvement = initial_solution.cost - best.cost
            improvement_pct = 100 * improvement / initial_solution.cost
            print(f"\nImprovement from Tabu Search:")
            print(f"  Absolute: {improvement:.2f}")
            print(f"  Relative: {improvement_pct:.2f}%")
        
        return best
    
    def _generate_neighbor(self, solution: Solution) -> Optional[Solution]:
        """
        Génère un voisin aléatoire
        """
        if len(solution.routes) == 0:
            return None
        
        neighbor = solution.copy()
        
        # Choisir un type de mouvement aléatoire
        move_type = random.choice(['relocate', 'swap', '2opt', 'reverse'])
        
        if move_type == 'relocate' and len(neighbor.routes) >= 2:
            # Relocate: déplacer un client
            route1_idx = random.randint(0, len(neighbor.routes) - 1)
            if not neighbor.routes[route1_idx]:
                return None
            
            pos = random.randint(0, len(neighbor.routes[route1_idx]) - 1)
            customer = neighbor.routes[route1_idx].pop(pos)
            
            route2_idx = random.randint(0, len(neighbor.routes) - 1)
            insert_pos = random.randint(0, len(neighbor.routes[route2_idx]))
            neighbor.routes[route2_idx].insert(insert_pos, customer)
            
            neighbor.routes = [r for r in neighbor.routes if r]
        
        elif move_type == 'swap' and len(neighbor.routes) >= 2:
            # Swap: échanger deux clients
            idx1, idx2 = random.sample(range(len(neighbor.routes)), 2)
            if not neighbor.routes[idx1] or not neighbor.routes[idx2]:
                return None
            
            pos1 = random.randint(0, len(neighbor.routes[idx1]) - 1)
            pos2 = random.randint(0, len(neighbor.routes[idx2]) - 1)
            
            neighbor.routes[idx1][pos1], neighbor.routes[idx2][pos2] = \
                neighbor.routes[idx2][pos2], neighbor.routes[idx1][pos1]
        
        elif move_type == '2opt' and len(neighbor.routes) > 0:
            # 2-opt intra-route
            route_idx = random.randint(0, len(neighbor.routes) - 1)
            if len(neighbor.routes[route_idx]) < 4:
                return None
            
            i = random.randint(0, len(neighbor.routes[route_idx]) - 3)
            j = random.randint(i + 2, len(neighbor.routes[route_idx]) - 1)
            
            neighbor.routes[route_idx][i+1:j+1] = list(reversed(neighbor.routes[route_idx][i+1:j+1]))
        
        elif move_type == 'reverse' and len(neighbor.routes) > 0:
            # Inverser une route complète
            route_idx = random.randint(0, len(neighbor.routes) - 1)
            neighbor.routes[route_idx] = list(reversed(neighbor.routes[route_idx]))
        
        # Vérifier faisabilité
        feasible = True
        for route in neighbor.routes:
            if route and not _is_route_time_feasible(self.instance, route):
                feasible = False
                break
        
        if not feasible:
            return None
        
        return neighbor


print("Simulated Annealing class loaded")

Simulated Annealing class loaded


---
## 7. SIMULATED ANNEALING (Refinement)

In [35]:
# ============================================================================
# TEST COMPLET: TABU SEARCH + SIMULATED ANNEALING
# ============================================================================

print("=" * 70)
print(f"HYBRID APPROACH ON {instance_obj.name}")
print("=" * 70)

# CREATE INITIAL SOLUTION (for comparison)
print("\n🔧 Creating initial solution...")
initial_solution = create_initial_solution(instance_obj)
print(f"Initial solution: {len(initial_solution.routes)} routes, cost={initial_solution.cost:.2f}")

# PHASE 1: TABU SEARCH
print("\n" + "=" * 70)
print("PHASE 1: TABU SEARCH")
print("=" * 70)

tabu_solver = TabuSearch(
    instance=instance_obj,
    max_iterations=500,       # Itérations Tabu Search
    max_time=120,             # Temps max (secondes)
    tabu_tenure=10,           # Durée dans liste taboue
    neighborhood_size=100     # Voisins explorés par itération
)

solution_tabu = tabu_solver.solve(verbose=True)

# PHASE 2: SIMULATED ANNEALING (amélioration)
print("\n" + "=" * 70)
print("PHASE 2: SIMULATED ANNEALING REFINEMENT")
print("=" * 70)

sa_solver = SimulatedAnnealing(
    instance=instance_obj,
    max_iterations=500,          # Itérations SA
    temperature_init=100.0,      # Température initiale
    cooling_rate=0.995,          # Taux refroidissement
    max_time=60                  # Temps max (secondes)
)

solution_final = sa_solver.improve(solution_tabu, verbose=True)

# RÉSULTATS FINAUX
print("\n" + "=" * 70)
print("FINAL RESULTS")
print("=" * 70)
print(f"\nInstance: {instance_obj.name}")
print(f"Customers: {instance_obj.num_customers}")

print(f"\n--- Solution Progression ---")
print(f"Initial (simple):    {initial_solution.cost:.2f}")
print(f"After Tabu Search:   {solution_tabu.distance:.2f}")
print(f"After SA refinement: {solution_final.distance:.2f}")

print(f"\nFinal Solution:")
print(f"  Total cost: {solution_final.cost:.2f}")
print(f"  Distance: {solution_final.distance:.2f}")
print(f"  Number of routes: {len(solution_final.routes)}")
print(f"  Feasible: {solution_final.is_feasible()}")
print(f"  Time violations: {solution_final.time_violations}")

if optimal_cost:
    gap = 100 * (solution_final.distance - optimal_cost) / optimal_cost
    print(f"\nComparison with optimal:")
    print(f"  Optimal cost: {optimal_cost:.2f}")
    print(f"  Our cost: {solution_final.distance:.2f}")
    print(f"  Gap: {gap:.2f}%")
    
    if gap < 5:
        print(f"  Status: ⭐ EXCELLENT (< 5%)")
    elif gap < 7:
        print(f"  Status: ✓ VERY GOOD (< 7%)")
    elif gap < 10:
        print(f"  Status: ✓ GOOD (< 10%)")
    else:
        print(f"  Status: ⚠ To improve (> 10%)")

print(f"\n✅ Test completed successfully!")

# Afficher quelques routes
print(f"\n--- Sample Routes (first 5) ---")
for i, route in enumerate(solution_final.routes[:5]):
    total_dist = solution_final._route_distance(route)
    print(f"Route {i+1}: {len(route)} customers, distance={total_dist:.2f}")
    print(f"  Clients: {route}")

HYBRID APPROACH ON C101.txt

🔧 Creating initial solution...
🔧 Génération solution initiale (routes individuelles)...
✅ Solution initiale créée: 100 routes, coût=5770.96
Initial solution: 100 routes, cost=5770.96

PHASE 1: TABU SEARCH

----------------------------------------------------------------------
TABU SEARCH OPTIMIZATION
----------------------------------------------------------------------
🔧 Génération solution initiale (routes individuelles)...
✅ Solution initiale créée: 100 routes, coût=5770.96

Starting Tabu Search from initial solution
  Initial cost: 5770.96
  Initial routes: 100

Parameters:
  Max iterations: 500
  Max time: 120s
  Tabu tenure: 10
  Neighborhood size: 100

Optimizing...
  Iter    0 | New best: 5733.67 | Routes: 99
  Iter    1 | New best: 5730.30 | Routes: 99
  Iter    2 | New best: 5693.00 | Routes: 98
  Iter    3 | New best: 5658.12 | Routes: 97
  Iter    4 | New best: 5626.61 | Routes: 96
  Iter    5 | New best: 5596.35 | Routes: 95
  Iter    7 | New b

---
## 8. TEST COMPLET: TABU SEARCH + SIMULATED ANNEALING

### Résolution Hybride

Nous appliquons maintenant l'approche hybride complète :
1. **Tabu Search** pour construire une bonne solution
2. **Simulated Annealing** pour raffiner cette solution

In [36]:
# ============================================================================
# TEST COMPLET: TABU SEARCH + SIMULATED ANNEALING
# ============================================================================

print("=" * 70)
print(f"HYBRID APPROACH ON {instance_obj.name}")
print("=" * 70)

# PHASE 1: TABU SEARCH
print("\n" + "=" * 70)
print("PHASE 1: TABU SEARCH")
print("=" * 70)

tabu_solver = TabuSearch(
    instance=instance_obj,
    max_iterations=500,       # Itérations Tabu Search
    max_time=120,             # Temps max (secondes)
    tabu_tenure=10,           # Durée dans liste taboue
    neighborhood_size=100     # Voisins explorés par itération
)

solution_tabu = tabu_solver.solve(verbose=True)

# PHASE 2: SIMULATED ANNEALING (amélioration)
print("\n" + "=" * 70)
print("PHASE 2: SIMULATED ANNEALING REFINEMENT")
print("=" * 70)

sa_solver = SimulatedAnnealing(
    instance=instance_obj,
    max_iterations=500,          # Itérations SA
    temperature_init=100.0,      # Température initiale
    cooling_rate=0.995,          # Taux refroidissement
    max_time=60                  # Temps max (secondes)
)

solution_final = sa_solver.improve(solution_tabu, verbose=True)

# RÉSULTATS FINAUX
print("\n" + "=" * 70)
print("FINAL RESULTS")
print("=" * 70)
print(f"\nInstance: {instance_obj.name}")
print(f"Customers: {instance_obj.num_customers}")

print(f"\n--- Solution Progression ---")
print(f"After Tabu Search:   {solution_tabu.distance:.2f}")
print(f"After SA refinement: {solution_final.distance:.2f}")

print(f"\nFinal Solution:")
print(f"  Total cost: {solution_final.cost:.2f}")
print(f"  Distance: {solution_final.distance:.2f}")
print(f"  Number of routes: {len(solution_final.routes)}")
print(f"  Feasible: {solution_final.is_feasible()}")
print(f"  Time violations: {solution_final.time_violations}")

if optimal_cost:
    gap = 100 * (solution_final.distance - optimal_cost) / optimal_cost
    print(f"\nComparison with optimal:")
    print(f"  Optimal cost: {optimal_cost:.2f}")
    print(f"  Our cost: {solution_final.distance:.2f}")
    print(f"  Gap: {gap:.2f}%")
    
    if gap < 5:
        print(f"  Status: ⭐ EXCELLENT (< 5%)")
    elif gap < 7:
        print(f"  Status: ✓ VERY GOOD (< 7%)")
    elif gap < 10:
        print(f"  Status: ✓ GOOD (< 10%)")
    else:
        print(f"  Status: ⚠ To improve (> 10%)")

print(f"\n✅ Test completed successfully!")

# Afficher quelques routes
print(f"\n--- Sample Routes (first 5) ---")
for i, route in enumerate(solution_final.routes[:5]):
    total_dist = solution_final._route_distance(route)
    print(f"Route {i+1}: {len(route)} customers, distance={total_dist:.2f}")
    print(f"  Clients: {route}")

HYBRID APPROACH ON C101.txt

PHASE 1: TABU SEARCH

----------------------------------------------------------------------
TABU SEARCH OPTIMIZATION
----------------------------------------------------------------------
🔧 Génération solution initiale (routes individuelles)...
✅ Solution initiale créée: 100 routes, coût=5770.96

Starting Tabu Search from initial solution
  Initial cost: 5770.96
  Initial routes: 100

Parameters:
  Max iterations: 500
  Max time: 120s
  Tabu tenure: 10
  Neighborhood size: 100

Optimizing...
  Iter    0 | New best: 5733.67 | Routes: 99
  Iter    1 | New best: 5730.30 | Routes: 99
  Iter    2 | New best: 5693.00 | Routes: 98
  Iter    3 | New best: 5658.12 | Routes: 97
  Iter    4 | New best: 5626.61 | Routes: 96
  Iter    5 | New best: 5596.35 | Routes: 95
  Iter    7 | New best: 5594.54 | Routes: 95
  Iter    8 | New best: 5591.90 | Routes: 95
  Iter    9 | New best: 5559.82 | Routes: 94
  Iter    9 | New best: 5559.82 | Routes: 94
  Iter  100 | VND impro

---
## 9. ANALYSE DES RÉSULTATS

### Interprétation

Le solveur hybride **Tabu Search + Simulated Annealing** a été appliqué avec succès sur l'instance VRPTW.

**Points clés :**
- ✅ **Tabu Search** : exploration systématique et construction de solutions
- ✅ **Simulated Annealing** : raffinement et optimisation fine
- ✅ **VND** : intensification locale périodique
- ✅ **Pas de contrainte de capacité** : focus uniquement sur les fenêtres temporelles

### Prochaines étapes possibles

1. **Tester sur d'autres instances** : R101, RC101, etc.
2. **Ajuster les paramètres** : tabu_tenure, temperature, iterations
3. **Runs multiples** : exécuter 20 fois pour statistiques robustes
4. **Benchmark complet** : comparer avec solutions optimales VRPLIB
5. **Visualisation** : tracer les routes sur un graphique

---
## 10. VALIDATION AVEC BENCHMARK (20 RUNS)

### Expérimentation Multi-Runs

Pour valider la robustesse du solveur, on effectue 20 runs sur une instance et on calcule les statistiques :
- Coût moyen, médian, min, max
- Écart-type
- Gap moyen par rapport à l'optimal
- Taux de faisabilité

In [37]:
def run_benchmark(instance_name: str, 
                  n_runs: int = 20,
                  max_iterations_ts: int = 500,
                  max_time_ts: float = 120,
                  max_iterations_sa: int = 500,
                  max_time_sa: float = 60) -> pd.DataFrame:
    """
    Exécute n_runs sur une instance et retourne les statistiques
    
    Args:
        instance_name: Nom de l'instance (ex: "C101.txt")
        n_runs: Nombre d'exécutions
        max_iterations_ts: Itérations max Tabu Search
        max_time_ts: Temps max Tabu Search
        max_iterations_sa: Itérations max Simulated Annealing
        max_time_sa: Temps max Simulated Annealing
    
    Returns:
        DataFrame avec les résultats de chaque run
    """
    print("=" * 70)
    print(f"BENCHMARK: {instance_name}")
    print("=" * 70)
    print(f"Runs: {n_runs}")
    print(f"Tabu Search: {max_iterations_ts} iter, {max_time_ts}s max")
    print(f"Simulated Annealing: {max_iterations_sa} iter, {max_time_sa}s max")
    print()
    
    # Charger l'instance
    instance_data = load_instance(instance_name)
    instance = VRPInstance(instance_data)
    
    # Charger solution optimale
    sol_name = instance_name.replace('.txt', '.sol').replace('.vrp', '.sol')
    optimal_cost = load_solution(sol_name)
    
    print(f"Instance: {instance.name}")
    print(f"  Customers: {instance.num_customers}")
    if optimal_cost:
        print(f"  Optimal cost: {optimal_cost:.2f}")
    print()
    
    results = []
    
    for run in range(n_runs):
        print(f"Run {run + 1:2d}/{n_runs}...", end=" ", flush=True)
        
        # Changer la seed pour chaque run
        random.seed(42 + run)
        np.random.seed(42 + run)
        
        start_time = time.time()
        
        # Phase 1: Tabu Search
        tabu_solver = TabuSearch(
            instance=instance,
            max_iterations=max_iterations_ts,
            max_time=max_time_ts,
            tabu_tenure=10,
            neighborhood_size=100
        )
        solution_tabu = tabu_solver.solve(verbose=False)
        
        # Phase 2: Simulated Annealing
        sa_solver = SimulatedAnnealing(
            instance=instance,
            max_iterations=max_iterations_sa,
            temperature_init=100.0,
            cooling_rate=0.995,
            max_time=max_time_sa
        )
        solution_final = sa_solver.improve(solution_tabu, verbose=False)
        
        elapsed = time.time() - start_time
        
        # Calculer le gap
        gap = None
        if optimal_cost and optimal_cost > 0:
            gap = 100 * (solution_final.distance - optimal_cost) / optimal_cost
        
        # Stocker les résultats
        results.append({
            'run': run + 1,
            'cost': solution_final.distance,
            'routes': len(solution_final.routes),
            'time': elapsed,
            'feasible': solution_final.is_feasible(),
            'time_violations': solution_final.time_violations,
            'gap': gap
        })
        
        status = "✓" if solution_final.is_feasible() else "✗"
        print(f"{status} Cost={solution_final.distance:.2f}, Gap={gap:.2f}% ({elapsed:.1f}s)")
    
    df = pd.DataFrame(results)
    
    # Afficher statistiques globales
    print()
    print("=" * 70)
    print("STATISTIQUES GLOBALES")
    print("=" * 70)
    print(f"\nCoûts:")
    print(f"  Meilleur:  {df['cost'].min():.2f}")
    print(f"  Moyenne:   {df['cost'].mean():.2f}")
    print(f"  Médiane:   {df['cost'].median():.2f}")
    print(f"  Pire:      {df['cost'].max():.2f}")
    print(f"  Écart-type: {df['cost'].std():.2f}")
    
    if optimal_cost:
        print(f"\nGaps (%):")
        print(f"  Meilleur:  {df['gap'].min():.2f}%")
        print(f"  Moyenne:   {df['gap'].mean():.2f}%")
        print(f"  Médiane:   {df['gap'].median():.2f}%")
        print(f"  Pire:      {df['gap'].max():.2f}%")
        
        # Évaluation qualité
        avg_gap = df['gap'].mean()
        if avg_gap < 5:
            quality = "⭐ EXCELLENT"
        elif avg_gap < 7:
            quality = "✓ TRÈS BON"
        elif avg_gap < 10:
            quality = "✓ BON"
        else:
            quality = "⚠ À AMÉLIORER"
        print(f"\n  Qualité: {quality}")
    
    print(f"\nTemps:")
    print(f"  Moyenne:   {df['time'].mean():.2f}s")
    print(f"  Médiane:   {df['time'].median():.2f}s")
    
    print(f"\nFaisabilité:")
    feasible_count = df['feasible'].sum()
    print(f"  Solutions faisables: {feasible_count}/{n_runs} ({100*feasible_count/n_runs:.0f}%)")
    
    return df


print("Benchmark function loaded")

Benchmark function loaded


### Lancement du Benchmark



In [ ]:
# ============================================================================
# BENCHMARK: 20 RUNS
# ============================================================================

# Lancer 20 runs sur l'instance actuelle
benchmark_results = run_benchmark(
    instance_name=instance_name,
    n_runs=20,
    max_iterations_ts=500,
    max_time_ts=120,
    max_iterations_sa=500,
    max_time_sa=60
)

# Afficher le DataFrame complet
print("\n" + "=" * 70)
print("DÉTAILS DES 20 RUNS")
print("=" * 70)
print(benchmark_results.to_string(index=False))

BENCHMARK: C101.txt
Runs: 20
Tabu Search: 500 iter, 120s max
Simulated Annealing: 500 iter, 60s max

Instance: C101.txt
  Customers: 100
  Optimal cost: 827.30

Run  1/20... 

🔧 Génération solution initiale (routes individuelles)...
✅ Solution initiale créée: 100 routes, coût=5770.96
✓ Cost=895.52, Gap=8.25% (61.5s)
Run  2/20... ✓ Cost=895.52, Gap=8.25% (61.5s)
Run  2/20... 🔧 Génération solution initiale (routes individuelles)...
✅ Solution initiale créée: 100 routes, coût=5770.96
🔧 Génération solution initiale (routes individuelles)...
✅ Solution initiale créée: 100 routes, coût=5770.96
✓ Cost=895.52, Gap=8.25% (62.1s)
Run  3/20... ✓ Cost=895.52, Gap=8.25% (62.1s)
Run  3/20... 🔧 Génération solution initiale (routes individuelles)...
✅ Solution initiale créée: 100 routes, coût=5770.96
🔧 Génération solution initiale (routes individuelles)...
✅ Solution initiale créée: 100 routes, coût=5770.96
✓ Cost=860.43, Gap=4.00% (62.1s)
Run  4/20... ✓ Cost=860.43, Gap=4.00% (62.1s)
Run  4/20... 🔧 Génération solution initiale (routes individuelles)...
✅ Solution initiale créée: 100 routes, coût=5770.96
🔧 Génération solution initiale (routes individuelles)...
✅ Solution in

---
## 11. VISUALISATION DES ROUTES

### Fonction de Visualisation

Créons une fonction pour visualiser les routes sur un graphique 2D :
- Dépôt en rouge (carré)
- Clients en bleu (cercles)
- Routes avec des couleurs différentes
- Affichage des distances

In [ ]:
def plot_routes(instance: VRPInstance, solution: Solution, title: str = "VRPTW Solution"):
    """
    Visualise les routes sur un graphique 2D
    
    Args:
        instance: Instance VRP
        solution: Solution à visualiser
        title: Titre du graphique
    """
    plt.figure(figsize=(14, 10))
    
    # Palette de couleurs
    colors = plt.cm.tab20(np.linspace(0, 1, len(solution.routes)))
    
    # 1. Tracer toutes les routes
    for route_idx, route in enumerate(solution.routes):
        if not route:
            continue
        
        # Coordonnées de la route (dépôt -> clients -> dépôt)
        x_coords = [instance.depot.x]
        y_coords = [instance.depot.y]
        
        for customer_id in route:
            customer = instance.customers[customer_id]
            x_coords.append(customer.x)
            y_coords.append(customer.y)
        
        # Retour au dépôt
        x_coords.append(instance.depot.x)
        y_coords.append(instance.depot.y)
        
        # Tracer la route
        plt.plot(x_coords, y_coords, '-', color=colors[route_idx], 
                linewidth=1.5, alpha=0.7, label=f'Route {route_idx+1}')
        
        # Marquer les clients de cette route
        for i, customer_id in enumerate(route):
            customer = instance.customers[customer_id]
            plt.plot(customer.x, customer.y, 'o', color=colors[route_idx], 
                    markersize=8, markeredgecolor='black', markeredgewidth=0.5)
            
            # Annoter quelques clients (pas tous pour éviter surcharge)
            if len(solution.routes) <= 15 or i % 2 == 0:
                plt.annotate(f'{customer_id}', (customer.x, customer.y),
                           fontsize=7, ha='center', va='center')
    
    # 2. Tracer le dépôt (en dernier pour qu'il soit visible)
    plt.plot(instance.depot.x, instance.depot.y, 's', color='red', 
            markersize=15, label='Dépôt', markeredgecolor='black', markeredgewidth=2)
    
    # 3. Configuration du graphique
    plt.xlabel('X', fontsize=12)
    plt.ylabel('Y', fontsize=12)
    plt.title(f'{title}\nCost: {solution.distance:.2f} | Routes: {len(solution.routes)} | Feasible: {solution.is_feasible()}', 
             fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # Légende (limiter si trop de routes)
    if len(solution.routes) <= 15:
        plt.legend(loc='best', fontsize=8, ncol=2)
    else:
        # Juste montrer dépôt dans la légende
        plt.legend(['Dépôt'], loc='best', fontsize=10)
    
    plt.tight_layout()
    plt.show()


print("Route visualization function loaded")

### Visualisation de la Solution Finale

Visualisons la meilleure solution obtenue avec Tabu Search + SA.

In [ ]:
# ============================================================================
# VISUALISATION DE LA SOLUTION FINALE
# ============================================================================

# Visualiser la solution finale (du test section 8)
plot_routes(instance_obj, solution_final, 
           title=f"Solution VRPTW - {instance_obj.name} (Tabu Search + SA)")

---
## 12. CONVERGENCE DE L'ALGORITHME

### Graphique de Convergence

Visualisons l'évolution du coût pendant l'optimisation pour analyser la convergence.

In [ ]:
def plot_convergence(tabu_solver: TabuSearch, sa_solver: SimulatedAnnealing, 
                     instance_name: str, optimal_cost: Optional[float] = None):
    """
    Visualise la convergence de l'algorithme hybride
    
    Args:
        tabu_solver: Solveur Tabu Search (avec historique)
        sa_solver: Solveur Simulated Annealing (avec historique)
        instance_name: Nom de l'instance
        optimal_cost: Coût optimal si connu
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # 1. Convergence Tabu Search
    ax1 = axes[0, 0]
    if len(tabu_solver.cost_history) > 0:
        ax1.plot(tabu_solver.cost_history, 'b-', alpha=0.5, linewidth=0.8, label='Coût courant')
        ax1.plot(tabu_solver.best_cost_history, 'r-', linewidth=2, label='Meilleur coût')
        
        if optimal_cost:
            ax1.axhline(y=optimal_cost, color='g', linestyle='--', linewidth=2, label='Optimal')
        
        ax1.set_xlabel('Itération')
        ax1.set_ylabel('Coût')
        ax1.set_title('Convergence Tabu Search')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
    
    # 2. Convergence Simulated Annealing
    ax2 = axes[0, 1]
    if len(sa_solver.cost_history) > 0:
        ax2.plot(sa_solver.cost_history, 'b-', alpha=0.5, linewidth=0.8, label='Coût courant')
        ax2.plot(sa_solver.best_cost_history, 'r-', linewidth=2, label='Meilleur coût')
        
        if optimal_cost:
            ax2.axhline(y=optimal_cost, color='g', linestyle='--', linewidth=2, label='Optimal')
        
        ax2.set_xlabel('Itération')
        ax2.set_ylabel('Coût')
        ax2.set_title('Convergence Simulated Annealing')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    # 3. Convergence globale (TS + SA)
    ax3 = axes[1, 0]
    if len(tabu_solver.best_cost_history) > 0 and len(sa_solver.best_cost_history) > 0:
        # Concaténer les historiques
        ts_len = len(tabu_solver.best_cost_history)
        sa_len = len(sa_solver.best_cost_history)
        
        iterations_ts = list(range(ts_len))
        iterations_sa = list(range(ts_len, ts_len + sa_len))
        
        # Tabu Search phase
        ax3.plot(iterations_ts, tabu_solver.best_cost_history, 'b-', linewidth=2, label='Tabu Search')
        
        # Simulated Annealing phase
        ax3.plot(iterations_sa, sa_solver.best_cost_history, 'orange', linewidth=2, label='Simulated Annealing')
        
        # Ligne de séparation
        ax3.axvline(x=ts_len, color='gray', linestyle='--', linewidth=1, alpha=0.5)
        ax3.text(ts_len, ax3.get_ylim()[1]*0.95, 'TS → SA', ha='center', fontsize=10)
        
        if optimal_cost:
            ax3.axhline(y=optimal_cost, color='g', linestyle='--', linewidth=2, label='Optimal')
        
        ax3.set_xlabel('Itération totale')
        ax3.set_ylabel('Meilleur coût')
        ax3.set_title(f'Convergence Globale - {instance_name}')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
    
    # 4. Distribution des coûts TS vs SA
    ax4 = axes[1, 1]
    if len(tabu_solver.best_cost_history) > 0 and len(sa_solver.best_cost_history) > 0:
        final_cost_ts = tabu_solver.best_cost_history[-1]
        final_cost_sa = sa_solver.best_cost_history[-1]
        
        phases = ['Tabu Search', 'Simulated\nAnnealing']
        costs = [final_cost_ts, final_cost_sa]
        colors_bar = ['blue', 'orange']
        
        bars = ax4.bar(phases, costs, color=colors_bar, alpha=0.7, edgecolor='black', linewidth=2)
        
        # Ajouter les valeurs sur les barres
        for i, (phase, cost) in enumerate(zip(phases, costs)):
            ax4.text(i, cost + max(costs)*0.02, f'{cost:.2f}', 
                    ha='center', va='bottom', fontsize=12, fontweight='bold')
        
        # Ligne optimale
        if optimal_cost:
            ax4.axhline(y=optimal_cost, color='g', linestyle='--', linewidth=2, label=f'Optimal: {optimal_cost:.2f}')
            ax4.legend()
        
        ax4.set_ylabel('Coût')
        ax4.set_title('Coûts Finaux par Phase')
        ax4.grid(True, axis='y', alpha=0.3)
        
        # Calculer amélioration
        improvement = final_cost_ts - final_cost_sa
        improvement_pct = 100 * improvement / final_cost_ts
        ax4.text(0.5, min(costs)*1.1, f'Amélioration SA:\n{improvement:.2f} ({improvement_pct:.2f}%)',
                ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()


print("Convergence plot function loaded")

### Visualisation de la Convergence

Affichons l'évolution du coût pendant l'optimisation (Tabu Search puis SA).

In [ ]:
# ============================================================================
# GRAPHIQUE DE CONVERGENCE
# ============================================================================

# Visualiser la convergence (utilise les solveurs de la section 8)
plot_convergence(tabu_solver, sa_solver, instance_obj.name, optimal_cost)

---
## 13. ANALYSE STATISTIQUE DES RÉSULTATS

### Graphiques de Distribution

Analysons la distribution des résultats sur les 20 runs avec des graphiques statistiques.

In [ ]:
def plot_benchmark_analysis(df: pd.DataFrame, instance_name: str, optimal_cost: Optional[float] = None):
    """
    Visualise l'analyse statistique des 20 runs
    
    Args:
        df: DataFrame avec les résultats des runs
        instance_name: Nom de l'instance
        optimal_cost: Coût optimal si connu
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # 1. Distribution des coûts (histogramme)
    ax1 = axes[0, 0]
    ax1.hist(df['cost'], bins=15, color='skyblue', edgecolor='black', alpha=0.7)
    ax1.axvline(df['cost'].mean(), color='red', linestyle='--', linewidth=2, label=f'Moyenne: {df["cost"].mean():.2f}')
    ax1.axvline(df['cost'].median(), color='green', linestyle='--', linewidth=2, label=f'Médiane: {df["cost"].median():.2f}')
    
    if optimal_cost:
        ax1.axvline(optimal_cost, color='orange', linestyle='--', linewidth=2, label=f'Optimal: {optimal_cost:.2f}')
    
    ax1.set_xlabel('Coût')
    ax1.set_ylabel('Fréquence')
    ax1.set_title(f'Distribution des Coûts - {instance_name}')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Évolution des coûts par run
    ax2 = axes[0, 1]
    ax2.plot(df['run'], df['cost'], 'bo-', linewidth=1.5, markersize=6)
    ax2.axhline(df['cost'].mean(), color='red', linestyle='--', linewidth=2, alpha=0.7, label='Moyenne')
    
    if optimal_cost:
        ax2.axhline(optimal_cost, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Optimal')
    
    ax2.set_xlabel('Run')
    ax2.set_ylabel('Coût')
    ax2.set_title('Coûts par Run')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Boxplot des gaps
    ax3 = axes[1, 0]
    if optimal_cost and 'gap' in df.columns:
        box = ax3.boxplot(df['gap'], vert=True, patch_artist=True)
        box['boxes'][0].set_facecolor('lightgreen')
        box['boxes'][0].set_alpha(0.7)
        
        # Ajouter statistiques
        ax3.text(1.15, df['gap'].median(), f"Médiane: {df['gap'].median():.2f}%", 
                va='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        ax3.text(1.15, df['gap'].mean(), f"Moyenne: {df['gap'].mean():.2f}%", 
                va='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
        
        ax3.set_ylabel('Gap (%)')
        ax3.set_title('Distribution des Gaps')
        ax3.set_xticklabels([instance_name])
        ax3.grid(True, axis='y', alpha=0.3)
    else:
        ax3.text(0.5, 0.5, 'Pas de solution optimale connue', 
                ha='center', va='center', fontsize=12)
        ax3.set_title('Distribution des Gaps')
    
    # 4. Temps d'exécution
    ax4 = axes[1, 1]
    ax4.bar(df['run'], df['time'], color='coral', alpha=0.7, edgecolor='black')
    ax4.axhline(df['time'].mean(), color='red', linestyle='--', linewidth=2, label=f'Moyenne: {df["time"].mean():.2f}s')
    
    ax4.set_xlabel('Run')
    ax4.set_ylabel('Temps (secondes)')
    ax4.set_title('Temps d\'Exécution par Run')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques textuelles
    print("\n" + "=" * 70)
    print("STATISTIQUES DÉTAILLÉES")
    print("=" * 70)
    print(f"\nCoûts:")
    print(f"  Min:        {df['cost'].min():.2f}")
    print(f"  Q1 (25%):   {df['cost'].quantile(0.25):.2f}")
    print(f"  Médiane:    {df['cost'].median():.2f}")
    print(f"  Q3 (75%):   {df['cost'].quantile(0.75):.2f}")
    print(f"  Max:        {df['cost'].max():.2f}")
    print(f"  Moyenne:    {df['cost'].mean():.2f}")
    print(f"  Écart-type: {df['cost'].std():.2f}")
    print(f"  CV:         {100*df['cost'].std()/df['cost'].mean():.2f}%")
    
    if optimal_cost and 'gap' in df.columns:
        print(f"\nGaps (%):")
        print(f"  Min:        {df['gap'].min():.2f}%")
        print(f"  Q1 (25%):   {df['gap'].quantile(0.25):.2f}%")
        print(f"  Médiane:    {df['gap'].median():.2f}%")
        print(f"  Q3 (75%):   {df['gap'].quantile(0.75):.2f}%")
        print(f"  Max:        {df['gap'].max():.2f}%")
        print(f"  Moyenne:    {df['gap'].mean():.2f}%")
        print(f"  Écart-type: {df['gap'].std():.2f}%")


print("Benchmark analysis plot function loaded")

### Visualisation de l'Analyse Statistique

Analysons les résultats des 20 runs avec des graphiques statistiques.

In [ ]:
# ============================================================================
# ANALYSE STATISTIQUE DES 20 RUNS
# ============================================================================

# Visualiser les statistiques des 20 runs (utilise benchmark_results de la section 10)
plot_benchmark_analysis(benchmark_results, instance_obj.name, optimal_cost)

In [ ]:
# ============================================================================
# ANALYSE STATISTIQUE DES 20 RUNS
# ============================================================================

# Analyser les résultats du benchmark (utilise benchmark_results de la section 10)
plot_benchmark_analysis(benchmark_results, instance_obj.name, optimal_cost)

---
## 14. CONCLUSION ET PERSPECTIVES

### Résumé du Projet

Ce notebook implémente un **solveur VRPTW hybride** combinant :
1. **Tabu Search** comme métaheuristique principale
2. **Simulated Annealing** pour le raffinement
3. **VND** pour l'intensification locale

### Points clés

✅ **Contraintes respectées** : Fenêtres temporelles uniquement (pas de capacité)  
✅ **Architecture modulaire** : Classes réutilisables et extensibles  
✅ **Validation robuste** : 20 runs pour statistiques fiables  
✅ **Visualisations complètes** : Routes, convergence, statistiques  

### Résultats attendus

Sur les instances Solomon (C101, R101, RC101) :
- **Gap moyen** : < 7% (objectif ADEME atteint)
- **Taux de faisabilité** : 100% des solutions respectent les fenêtres temporelles
- **Temps de calcul** : ~3 minutes par instance (TS + SA)

### Améliorations possibles

1. **Paramètres adaptatifs** : ajuster dynamiquement tabu_tenure et temperature
2. **Nouveaux opérateurs** : cross-exchange, Or-opt
3. **Parallélisation** : runs multiples en parallèle
4. **Instances plus grandes** : tester sur >1000 clients
5. **Flotte hétérogène** : intégrer différents types de véhicules

### Benchmarks VRPLIB

Pour validation complète, tester sur :
- **C1** (clustered, short time windows)
- **C2** (clustered, long time windows)
- **R1** (random, short time windows)
- **R2** (random, long time windows)
- **RC1** (mixed, short time windows)
- **RC2** (mixed, long time windows)